In [1]:
#based on https://stackoverflow.com/questions/43101497/how-do-stream-data-to-a-bokeh-plot-in-jupyter-with-a-high-refresh-rate

import time
import os
from threading import Thread, Event

from sh import tail

import numpy as np

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, show, push_notebook

# import panel as pn
# pn.extension()

In [2]:
output_notebook()

Loading BokehJS ...

In [4]:
fig = figure(
        height=800,
        width=1200,
        title="PowerSensor3 measurements",
        y_range=(0, 10),
    )
fig.xaxis.axis_label = "Time (s)"
fig.yaxis.axis_label = "Power (W)"


ps_data = ColumnDataSource(data=dict(time=[], power_ps=[]))

line_ps = fig.line("time", "power_ps", source=ps_data)

handle = show(fig, notebook_handle=True)

In [6]:
event = Event()

def blocking_callback(id, event):
    n_sec_to_show = 30
    # powersensor dumpefile freq reduced by a factor 1000: so 20 Hz
    ps_freq = 20
    n_show = n_sec_to_show * ps_freq

    # wait for file to exist
    fname = '/dev/shm/ps3.txt'
    timeout = 10  # seconds
    tstart = time.time()
    while not os.path.exists(fname):
        event.wait(.1)
        if time.time() - tstart > timeout:
            print(f"File {fname} still does not exist, giving up")
            return

    print("Starting read")

    # for lines in tail(open('/dev/shm/ps3.txt'), event):        
        # only lines that start with S are actual measurements
    for line in tail('-f', '/dev/shm/ps3.txt', _iter=True):
        if not line.startswith('S'):
            continue
        line = line.split(' ')
        t = float(line[1])
        p = float(line[-1])
        ps_data.stream({'time': [t], 'power_ps': [p]}, n_show)
        # entries = [line.split(' ') for line in lines if line.startswith('S')]
        # if not entries:
            # continue
        # timestamps = [np.average([float(entry[1]) for entry in entries])]
        # powers = [np.average([float(entry[-1]) for entry in entries])]

        # test_data.stream({'x': timestamps, 'y': powers}) #, n_show)

        push_notebook(handle=handle)
        # event.wait(period)
    print("exit")

thread = Thread(target=blocking_callback, args=(id, event))
thread.start()
# blocking_callback(id, event)

Starting read


In [16]:
thread.is_alive()

False

In [17]:
event.set()

In [18]:
thread.is_alive()

False